In [0]:
from google.colab import drive
drive.mount('/content/drive')
print('done')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
done


In [0]:
import pandas as pandas 
import numpy as numpy
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [0]:
data = pandas.read_csv('/content/drive/My Drive/SML/preprocessed_data1_28_04_19.csv', index_col=0)#, sep = '\t')
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation why the edits made under my userna...,0,0,0,0,0,0
1,d aww he matches this background colour i am s...,0,0,0,0,0,0
2,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [0]:
print(numpy.where(pandas.isnull(data)))
data['comment_text'].replace('', numpy.nan, inplace=True) # replace empty cell by nan number
data.dropna(subset=['comment_text'], inplace=True)
print(numpy.where(pandas.isnull(data)))

(array([ 4482,  6300, 17311, 52442, 53787, 61758, 82681]), array([0, 0, 0, 0, 0, 0, 0]))
(array([], dtype=int64), array([], dtype=int64))


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
vocab_size = 40000
tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(data['comment_text'])

maxLen = 250
sequences = tokenizer.texts_to_sequences(data['comment_text'])
dd = pad_sequences(sequences, maxlen=maxLen)

In [0]:
print(dd.shape)

(159564, 250)


In [0]:
# label
label = data[data.columns[1:]].values
print(label.shape)
print(label)

(159564, 6)
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [0]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [0]:
# from keras.layers.embeddings import Embedding

from keras.layers.embeddings import Embedding

from keras.layers import Flatten, Conv1D, MaxPooling1D, Dropout, Activation, GlobalMaxPooling1D

In [0]:
import keras_metrics
from sklearn.metrics import precision_score, recall_score

In [0]:
model_conv = Sequential()
model_conv.add(Embedding(vocab_size, 100, input_length=maxLen))

model_conv.add(Dropout(0.2))

model_conv.add(Conv1D(100, 6, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))

# model_conv.add(GlobalMaxPooling1D())
model_conv.add(Dropout(0.2))

model_conv.add(Conv1D(64, 6, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))

# model_conv.add(GlobalMaxPooling1D())
model_conv.add(Dropout(0.2))
# model_conv.add(Conv1D(32, 4, activation='relu'))
# model_conv.add(MaxPooling1D(pool_size=4))

# model_conv.add(Conv1D(32, 4, activation='relu'))
# model_conv.add(MaxPooling1D(pool_size=4))

model_conv.add(Flatten())
model_conv.add(Dense(64, activation='relu'))

model_conv.add(Dropout(0.2))
model_conv.add(Dense(64, activation='relu'))

model_conv.add(Dropout(0.2))
model_conv.add(Dense(1, activation='sigmoid'))

#model_conv.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model_conv.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.1), metrics=['accuracy', keras_metrics.precision(), keras_metrics.recall()])#metrics=['accuracy'])
# model_conv.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.1), metrics=[keras_metrics.recall()])#metrics=['accuracy'])

# model_conv.fit(padded_docs, label, epochs=2, validation_split=.2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
col = data.columns[1:]
print(col)

NameError: ignored

In [0]:
# model_nn.fit(train, label, epochs=2, validation_split=.2)
for i in range(6):
    #print(label[:,i:].shape)
    #print(label[:,i:])
    print(col[i])
    print("-------------------------------------------------------------------------------------------------------------------")
    l = label[:,i].reshape((len(label), 1))
    print(l.shape)
    
    x_train, x_test, y_train, y_test = train_test_split(dd, l, test_size=0.3, random_state=0)
    
    x_train_smote, y_train_smote = smote.fit_sample(x_train, y_train)

    
    
    #model_conv.fit(dd, l, batch_size=512, epochs=2, validation_split=.3)
    model_conv.fit(x_train_smote, y_train_smote, batch_size=512, epochs=10, validation_data=(x_test, y_test))
    #break

toxic
-------------------------------------------------------------------------------------------------------------------
(159564, 1)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train on 201920 samples, validate on 47870 samples
Epoch 1/10
201920/201920 [==============================] - 1439s 7ms/step - loss: 7.9713 - acc: 0.5000 - precision: 0.5000 - recall: 1.0000 - val_loss: 14.4237 - val_acc: 0.0953 - val_precision: 0.0953 - val_recall: 1.0000
Epoch 2/10
201920/201920 [==============================] - 1436s 7ms/step - loss: 7.9711 - acc: 0.5000 - precision: 0.5000 - recall: 1.0000 - val_loss: 14.4237 - val_acc: 0.0953 - val_precision: 0.0953 - val_recall: 1.0000
Epoch 3/10
201920/201920 [==============================] - 1544s 8ms/step - loss: 7.9712 - acc: 0.5000 - precision: 0.5000 - recall: 1.0000 - val_loss: 14.4237 - val_acc: 0.0953 - val_precision: 0.0953 - val_recall: 1.0000
Epoch 4/10
  4608/201920 [..............................] - ETA: 27:42 - loss: 7.9850 - acc: 0.4991 - precision: 0.4991 - recall: 1.0000

In [0]:
# # model_nn.fit(train, label, epochs=2, validation_split=.2)
# for i in range(6):
#     #print(label[:,i:].shape)
#     #print(label[:,i:])
#     print(col[i])
#     print("-------------------------------------------------------------------------------------------------------------------")
#     l = label[:,i].reshape((len(label), 1))
#     print(l.shape)
    
#     x_train, x_test, y_train, y_test = train_test_split(dd, l, test_size=0.3, random_state=0)
    
#     x_train_smote, y_train_smote = smote.fit_sample(x_train, y_train)

    
    
#     #model_conv.fit(dd, l, batch_size=512, epochs=2, validation_split=.3)
#     model_conv.fit(x_train_smote, y_train_smote, batch_size=512, epochs=10, validation_data=(x_test, y_test))
#     #break

NameError: ignored

**rough below**